In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Zapoznajemy się z danymi

In [ ]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()

In [ ]:
type(boston_dataset)

In [ ]:
boston_dataset.keys()

In [ ]:
print(boston_dataset.DESCR)

In [ ]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston=
boston.head()

In [ ]:
plt.hist(boston_dataset.target, normed=True, bins=30)
plt.ylabel('Probability');

In [ ]:
for var in boston.columns:
    plt.scatter(boston_dataset.target,boston[var])
    plt.xlabel("Target")
    plt.ylabel(var)
    plt.show()

In [ ]:
# raczej o czystych, liniowych zależnościach nie ma mowy, ale zobaczmy co da się z tym zrobić...

# Dopasujmy pierwszy model, który będzie służył nam później za benchmark

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(boston,boston_dataset.target)

prediction = model.predict(boston)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import  mean_absolute_error
from sklearn.metrics import r2_score

def assess_model(prediction, target):
    rmse=np.sqrt(mean_squared_error(target, prediction))
    mae=mean_absolute_error(target, prediction)
    r2=r2_score(target, prediction)

    plt.scatter(target,prediction)
    plt.xlabel("Target")
    plt.ylabel("Prediction")
    plt.show()
    
    return {'RMSE: ':rmse,'MAE: ':mae,'R^2: ':r2}

In [ ]:
print(assess_model(prediction,boston_dataset.target))

# Spróbujmy to przebić:

In [ ]:
# jakie techniki możemy wykorzystać:
- usunięcie outlierów 
- normalizacja rozkładów predyktorów ((PowerTransformer) 
- inne przekształcenia w poszukiwaniu zmiennych nieliniowych: 
                                      - sklearn.compose.TransformedTargetRegressor, jeśli chcemy transformować target 
                                      - PolynomialFeatures jeśli chcemy szukać wielomianów 
                                      - inne transformacje (np.log(), np.exp()) 
- model regularyzowany 
- optymalizacja hiperparamtrów (regularyzacji, wielomianow) 

Ćwiczenie 1

Proszę stworzyć możliwie dobrze dopasowany model

hint: proszę użyć funkcji make_pipeline lub Pipeline

# Dopasowanie dopasowaniem, ale co z właściwościami predykcyjnymi?

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston,boston_dataset.target,test_size=100)

In [ ]:
model_poly = Pipeline([('polyfeatures',PolynomialFeatures(3,include_bias=False)),('lm',LinearRegression())])
model_poly.fit(X_train, y_train)
prediction_poly_test=model_poly.predict(X_test)

In [ ]:
assess_model(prediction_poly_test,y_test)

In [ ]:
# niestety to tylko overfit :(
# z czego wynika nagle tak duży rozrzut predykcji na skali?

In [ ]:
# --> LR Zoom-in

Ćwiczenie 3: zastosujmy to do danych rzeczywistych

proszę zbudować model wykorzystujący wielomiany do 3 stopnia, ale z wykorzystaniem regresji grzbietowej

poeksperymentujmy ręcznie z parametrem lambda- poszukajmy najlepszego

In [ ]:
from sklearn.linear_model import Ridge

# Co się stało z betami?

In [ ]:
def diagnose_betas(betas):
    plt.hist(betas)
    result={'Count of betas: ':len(betas),'Mean: ':np.mean(betas),'Standard deviation: ':np.std(betas)}
    return result

In [ ]:
diagnose_betas(model_poly.named_steps['lm'].coef_)

In [ ]:
diagnose_betas(model_ridge.named_steps['ridge'].coef_)

In [ ]:
# wnioski:
# mamy mniejsze bety = mniejszą wariancję modelu, ale co z tego, skoro on wcale nie jest lepszy...

In [ ]:
# --> prezentacja: oglądamy funkcję straty

Ćwiczenie 4:
    proszę zbudować model regresji grzbietowej w oparciu o wystandaryzowane zmienne

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# udało się poprawić model (być może)

# To lecimy z Lasso

In [ ]:
from sklearn.linear_model import Lasso

Ćwiczenie 5:
    proszę analogiczny model, tylko wykorzystując Lasso zamiast Ridge

# Elastic net

proszę zbudować możliwie dobrze dopasowany model wykorztująć regularyzację ElasticNet z parametrami alpha i l1_ratio

In [ ]:
from sklearn.linear_model import ElasticNet


# Grid search
Proszę zbudować model regresji grzbietowej wykorzytujący optymalna kombinację stopnia wielomianu i współczynnika alfa
Proszę uruchomić funkcję assess_model na tym modelu

In [ ]:
from sklearn.model_selection import GridSearchCV


# Raportowanie modelu

    Raport powinien zawierać następujące informacje:
    
    - zmienne, które weszły do modelu (ostatecznie obiekt powinien być możliwie prosty!)
    - wykresy zmienna oczekiwana - zmienna obserwowana
    - diagnostyka reszt
    - statystyki właściwości predykcyjnych (np. R^2, MAE, RMSE) w podziale na zbiór treningowy i testowy: estetyczna tabela
    - "historię optymalizacji"- jakie metody były stosowane, które 
    
    
    Ponad to:
    - zwykle w biznesie model budowany jest na podzbiorze obserwacji- należy podać ograniczenia i uzasadnić ich wybór
    - zastosowanie i ograniczenia (np. model nie jest przeznaczony do wyceny nieruchomości luksusowych)